In [1]:
import cv2
import mediapipe as mp
import glob
from ultralytics import YOLO
import math
import pandas as pd
import numpy as np

model = YOLO("yolo11x-pose.pt")
fall_images = glob.glob("fall_dataset/images/fall/*.jpg")
not_fall_images = glob.glob("fall_dataset/images/not-fall/*.jpg")

def append_dataset(df, input_images_path, fall_condition: bool):
    for image_path in input_images_path:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (416, 416))
        results = model(img)
        img = results[0].plot()

        keypoint = results[0].keypoints.data[0].numpy()

        # tidak terdeteksi manusia
        if len(keypoint) == 0:
            continue

        left_shoulder_y = keypoint[5][1]
        left_shoulder_x = keypoint[6][0]
        right_shoulder_y = keypoint[6][1]
        right_shoulder_x = keypoint[6][0]
        left_body_y = keypoint[11][1]
        left_body_x = keypoint[11][0]
        right_body_y = keypoint[12][1]
        right_body_x = keypoint[12][0]
        len_factor = math.sqrt(((left_shoulder_y - left_body_y) ** 2 + (left_shoulder_x - left_body_x) ** 2))

        left_knee_y = keypoint[13][1]
        left_knee_x = keypoint[13][0]
        right_knee_y = keypoint[14][1]
        right_knee_x = keypoint[14][0]
        
        left_foot_y = keypoint[15][1]
        right_foot_y = keypoint[16][1]

        data = {
            'left_shoulder_y': left_shoulder_y,
            'left_shoulder_x': left_shoulder_x,
            'right_shoulder_y': right_shoulder_y,
            'right_shoulder_x': right_shoulder_x,
            'left_body_y': left_body_y,
            'left_body_x': left_body_x,
            'right_body_y': right_body_y,
            'right_body_x': right_body_x,
            'len_factor': len_factor,
            'left_knee_y': left_knee_y,
            'left_knee_x': left_knee_x,
            'right_knee_y': right_knee_y,
            'right_knee_x': right_knee_x,
            'left_foot_y': left_foot_y,
            'right_foot_y': right_foot_y,
            'fall': fall_condition
        }
        temp = pd.DataFrame(data, index=[0])
        df = pd.concat([df, temp], ignore_index=True)

    return df

In [2]:
df = pd.DataFrame(columns=['left_shoulder_y', 'left_shoulder_x', 'right_shoulder_y', 'right_shoulder_x', 'left_body_y', 'left_body_x', 'right_body_y', 'right_body_x', 'len_factor', 'left_knee_y', 'left_knee_x', 'right_knee_y', 'right_knee_x', 'left_foot_y', 'right_foot_y', 'fall'])

df = append_dataset(df, fall_images, True)
df = append_dataset(df, not_fall_images, False)

df.head()
df.to_csv('ml/dataset.csv', index=False)


0: 640x640 3 persons, 660.4ms
Speed: 3.6ms preprocess, 660.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)



/var/folders/q3/rfd50l751570rjpc5f926_kc0000gn/T/ipykernel_5872/319104510.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)


0: 640x640 1 person, 563.3ms
Speed: 1.6ms preprocess, 563.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 583.1ms
Speed: 1.4ms preprocess, 583.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 550.7ms
Speed: 1.6ms preprocess, 550.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 550.5ms
Speed: 1.4ms preprocess, 550.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 546.8ms
Speed: 1.4ms preprocess, 546.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 547.8ms
Speed: 1.6ms preprocess, 547.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 547.4ms
Speed: 1.4ms preprocess, 547.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 553.8ms
Speed: 1.5ms preprocess, 553.8ms inference, 0.5ms postprocess per image a